In [1]:
import os
import pprint
import sys


path_cwd = os.getcwd()
print(path_cwd)
path_petsard = os.path.dirname(os.path.dirname(path_cwd))
print(path_petsard)
sys.path.append(path_petsard)

pp = pprint.PrettyPrinter(depth=2)

/Users/justyn.chen/Dropbox/310_Career_工作/20231016_NICS_資安院/41_PETsARD/PETsARD/demo/dev
/Users/justyn.chen/Dropbox/310_Career_工作/20231016_NICS_資安院/41_PETsARD/PETsARD


In [2]:
from PETsARD import (
    Loader,
    Splitter,
    Processor,
    Synthesizer,
    Describer,
)
from PETsARD.util import safe_round


load = Loader(
    filepath='benchmark://nist-ma2018',
    na_values='N',
)
load.load()
print(f"Raw data (Loader) # rn = {load.data.shape}")

split = Splitter(train_split_ratio=0.8, random_state=42)
split.split(data=load.data, metadata=load.metadata)
print(f"Split data (Splitter) # rn = {split.data[1]['train'].shape}")

proc = Processor(metadata=split.metadata)
proc.fit(data=split.data[1]['train'])
preproc_data = proc.transform(data=split.data[1]['train'])
print(f"Preproc data (Processor) # rn = {preproc_data.shape}")

syn = Synthesizer(method='smartnoise-pacsynth',
                  empty_value='test') # 'default')
syn.create(data=preproc_data, metadata=proc._metadata)
syn.fit_sample()
print(f"Syn data (Synthesizer) by Preproc data w/ Preproc Metadata # rn = {syn.data_syn.shape}")

postproc_data = proc.inverse_transform(data=syn.data_syn)
print(f"Postproc data (Processor) # rn = {postproc_data.shape}")

desc = Describer(config={'method': 'custom', 'describe': ['global_na_count']})
desc.create(data={'data': postproc_data})
desc.eval()

Loader - Benchmarker : Success download the benchmark dataset from https://petsard-benchmark.s3.amazonaws.com/ma2018.csv.
Raw data (Loader) # rn = (7244, 24)
Split data (Splitter) # rn = (5795, 24)
Preproc data (Processor) # rn = (3394, 24)
Synthesizer (SmartNoise): Fitting pacsynth.
Synthesizer (SmartNoise): Fitting pacsynth spent 1.8299 sec.
Synthesizer (SmartNoise): Sampling pacsynth # 3394 rows (same as input data) in 12.6149 sec.
Syn data (Synthesizer) by Preproc data w/ Preproc Metadata # rn = (3394, 24)


TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

In [ ]:
split.data[1]['train'].head(2)

,PUMA,AGEP,SEX,MSP,HISP,RAC1P,NOC,NPF,HOUSING_TYPE,OWN_RENT,...,PINCP,PINCP_DECILE,POVPIP,DVET,DREM,DPHY,DEYE,DEAR,PWGTP,WGTP
0,25-00703,28,1,6.0,0,1,NaN,NaN,3,0,...,35000.0,5.0,NaN,NaN,2.0,2.0,2,2,60,0
1,25-00703,20,2,6.0,0,1,NaN,NaN,3,0,...,3600.0,1.0,NaN,NaN,2.0,2.0,2,2,11,0


In [ ]:
preproc_data.head(2)

,PUMA,AGEP,SEX,MSP,HISP,RAC1P,NOC,NPF,HOUSING_TYPE,OWN_RENT,...,PINCP,PINCP_DECILE,POVPIP,DVET,DREM,DPHY,DEYE,DEAR,PWGTP,WGTP
0,0.992632,1.003049,0.96857,1.372267,-0.215142,-0.346568,-0.73299,-4.692435e-08,-0.259128,-0.246768,...,-0.41841,-0.286684,-1.312170,2.029254e-08,0.223311,0.251645,0.141658,0.191985,-0.228867,-0.158307
1,0.345944,0.286378,-1.03245,-0.887597,-0.215142,-0.346568,0.20843,-2.882458e-01,-0.259128,-0.246768,...,0.49533,1.064630,0.670438,2.029254e-08,0.223311,0.251645,0.141658,0.191985,0.088869,0.478387


In [ ]:
syn.data_syn.head(2)

,PUMA,AGEP,SEX,MSP,HISP,RAC1P,NOC,NPF,HOUSING_TYPE,OWN_RENT,...,PINCP,PINCP_DECILE,POVPIP,DVET,DREM,DPHY,DEYE,DEAR,PWGTP,WGTP
0,None,NaN,-1.03245,3.324075e-08,-0.215142,-0.346568,1.149849,0.531327,-0.259128,-0.246768,...,-4.868930e-08,3.535386e-09,0.670438,2.029254e-08,0.223311,0.251645,0.141658,0.191985,None,None
1,None,NaN,-1.03245,3.324075e-08,-0.215142,-0.346568,1.149849,0.531327,-0.259128,-0.246768,...,-4.868930e-08,3.535386e-09,0.670438,2.029254e-08,0.223311,0.251645,0.141658,0.191985,None,None
